# Train ReAct agent with code sandbox

In this tutorial, we will demonstrate how to train a [ReAct](https://arxiv.org/abs/2210.03629) agent to solve math problem with code sandbox.

The agent works as follows:
1. Given a math problem, the agent first query LLM to generate response and tool calls, which are python code to be executed in sandbox.
2. If there is a tool call, the agent execute the python code in code sandbox.
3. After code execution, the agent get the result from sandbox and append to chat history.
4. The agent query LLM again until no tool call or max context length reached.


<figure>
  <img src="https://langchain-ai.github.io/langgraph/agents/assets/agent.png" alt="ReAct" width="400">
  <figcaption style="font-style: italic; color: #666;">
    source: <a href="https://langchain-ai.github.io/langgraph/agents/overview/" target="_blank">LangGraph</a>
  </figcaption>
</figure>

## 1. Prerequisite

To run the examples in this notebook, you need to install the verl package first.
```bash
git clone https://github.com/volcengine/verl
cd verl
pip install -e .
```

In [1]:
import asyncio
import sys
import tempfile
import os
import socket
import json

import requests
import ray
import fastapi
import uvicorn
from starlette.requests import Request
from starlette.responses import JSONResponse
from pprint import pprint

import verl

ray.init(runtime_env={"env_vars": {"PYTORCH_CUDA_ALLOC_CONF": "expandable_segments:False", "VLLM_USE_V1": "1"}})
verl_config_dir = os.path.join(os.path.dirname(verl.__file__), "trainer/config")

[2025-12-12 06:40:30,045 I 389590 389590] gcs_rpc_client.h:648: successful connect gcs: 10.122.252.202:48885
[2025-12-12 06:40:30,389 I 389590 389590] gcs_rpc_client.h:648: successful connect gcs: 10.122.252.202:48885
[2025-12-12 06:40:32,889 I 389590 389590] gcs_rpc_client.h:648: successful connect gcs: 10.122.252.202:48885
[2025-12-12 06:40:32,891 I 389590 389590] gcs_rpc_client.h:648: successful connect gcs: 10.122.252.202:48885
2025-12-12 06:40:32,893	INFO worker.py:1887 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
[2025-12-12 06:40:32,894 I 389590 389590] gcs_rpc_client.h:648: successful connect gcs: 10.122.252.202:48885


Logs are printed to python-core-driver-01000000ffffffffffffffffffffffffffffffffffffffffffffffff_389590.log


For demo purpose, we will use Qwen/Qwen3-1.7B as the LLM. First, let's download required model and dataset used in this tutorial.

In [2]:
import pyarrow.parquet as pq
from huggingface_hub import snapshot_download


import os 

os.environ['http_proxy'] = 'http://sys-proxy-rd-relay.byted.org:8118'
os.environ['https_proxy'] = 'http://sys-proxy-rd-relay.byted.org:8118'
os.environ['no_proxy'] = 'code.byted.org'


snapshot_download(
    repo_id="verl-team/lighteval-MATH-preprocessed",
    repo_type="dataset",
    local_dir=os.path.expanduser("~/verl-team/lighteval-MATH-preprocessed"),
)
snapshot_download(
    repo_id="Qwen/Qwen3-1.7B",
    repo_type="model",
    local_dir=os.path.expanduser("~/Qwen/Qwen3-1.7B"),
)

model_path = os.path.expanduser("~/Qwen/Qwen3-1.7B")
train_file = os.path.expanduser("~/verl-team/lighteval-MATH-preprocessed/train.parquet")
test_file = os.path.expanduser("~/verl-team/lighteval-MATH-preprocessed/test.parquet")

test = pq.read_table(test_file)
test_file = os.path.expanduser("~/verl-team/lighteval-MATH-preprocessed/test_100.parquet")
pq.write_table(test[:100], test_file)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

verl support both vllm and sglang rollout server for high performance inference. This tutorial has been tested on both vllm and sglang, you can choose either of them to run the tutorial.

In [3]:
rollout_name = "vllm"  # vllm or sglang

## 2. Advanced tool call with code sandbox

Now, let's see a more realistic example of tool call with code sandbox, which is widely used in real-world applications.

### 2.1 Implement a naive code sandbox

To execute python code snippet generated by LLM, we need a code sandbox environment. In this tutorial, we will implement a very naive code sandbox, which is
a FastAPI http server with `/run_code` endpoint. The server works as follows:
1. Receive a http request, write the python code snippet to a temp file.
2. Spawn a subprocess to execute the code, and get stdout and stderr of the subprocess.
3. Return the stdout and stderr of the subprocess as http response.

> 🚨 **WARNING:** This naive code sandbox is for demonstration purpose only, do not use it in production. Please use docker/kata container for stronger isolation and security restriction.

In [4]:
from transformers.utils import get_json_schema
from verl.tools.base_tool import BaseTool, OpenAIFunctionToolSchema, ToolResponse


@ray.remote(num_cpus=1)
class Sandbox:
    """Sandbox to execute python code."""

    def __init__(self):
        self.address = ray._private.services.get_node_ip_address()
        self.port = self._get_free_port()
        asyncio.create_task(self._start_fastapi_server())

    async def code_execution(self, request: Request):
        request_json = await request.json()
        code = request_json["code"]
        # print(f"execute code:\n{code}")

        _, temp_file = tempfile.mkstemp(suffix=".py", prefix="temp_code", dir=None, text=True)
        with open(temp_file, "w") as f:
            f.write(code)

        try:
            process = await asyncio.create_subprocess_exec(
                sys.executable, temp_file, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
            )

            stdout, stderr = await process.communicate()

            response = {
                "status": "Success" if process.returncode == 0 else "Failed",
                "run_result": {
                    "status": "Finished",
                    "stdout": stdout.decode(),
                    "stderr": stderr.decode(),
                    "return_code": process.returncode,
                },
            }
            return JSONResponse(content=response)
        finally:
            try:
                os.unlink(temp_file)
            except Exception:
                pass

    def _get_free_port(self):
        with socket.socket() as sock:
            sock.bind(("", 0))
            return sock.getsockname()[1]

    async def _start_fastapi_server(self):
        app = fastapi.FastAPI()
        app.router.add_api_route("/run_code", self.code_execution, methods=["POST"])

        config = uvicorn.Config(app, host=["::", "0.0.0.0"], port=self.port, log_level="warning")
        server = uvicorn.Server(config)
        await server.serve()

    async def get_server_address(self) -> str:
        """Get FastAPI server address."""
        return f"{self.address}:{self.port}"

### 2.2 Define sandbox tool

As shown in the previous section, we also defined a tool for the code sandbox. In the `execute` method, we send the code snippet to code sandbox by http request and get the output.

In [5]:
import re
import aiohttp


class SandboxTool(BaseTool):
    def __init__(self, config: dict, tool_schema: OpenAIFunctionToolSchema):
        super().__init__(config, tool_schema)
        # Different model may use different code pattern, e.g. python, py, etc.
        self.code_pattern = re.compile(r"```py(.*?)```", re.DOTALL)

    async def code_interpreter(self, code: str) -> str:
        """Execute the code in the sandbox.

        Args:
            code: The code to be executed.

        Returns:
            str: The output of the code execution.
        """
        async with aiohttp.ClientSession() as session:
            async with session.post(
                self.config.get("sandbox_fusion_url"),
                json={"code": code},
            ) as resp:
                resp.raise_for_status()
                result = await resp.json()
                stdout, stderr = result["run_result"]["stdout"], result["run_result"]["stderr"]
                return stdout + stderr

    def get_openai_tool_schema(self) -> OpenAIFunctionToolSchema:
        schema = get_json_schema(self.code_interpreter)
        return OpenAIFunctionToolSchema(**schema)

    async def execute(self, instance_id: str, parameters: dict, **kwargs) -> tuple[str, float, dict]:
        code = parameters["code"]
        matches = self.code_pattern.findall(code)
        if matches:
            code = matches[0].strip()

        # NOTE: Some script may not explicitly print result, we need to add a print statement to the end of the script.
        # More better way is to SFT the model to make it print result by default, we skip SFT stage in this tutorial.
        lines = code.split("\n")
        for i, line in reversed(list(enumerate(lines))):
            if line == "":
                continue
            if not lines[i].startswith("print"):
                lines[i] = f"print({line})"
            break
        code = "\n".join(lines)

        result = await self.code_interpreter(code)
        return ToolResponse(text=result), 0.0, {}

## 3. End-to-end training with tool agent loop

After tool has been implemented and tested, we can do end-to-end RL training to tune the model to properly use the tool. To simplify agentic RL training, verl provide [Agent Loop](https://verl.readthedocs.io/en/latest/advance/agent_loop.html) abstraction, which allow user to define custom agent loop:
- Search agent
- Math agent
- SWE agent
- GUI agent
- ...

For ease of use, verl provide two pre-defined agent loop:
- SingleTurnAgentLoop: single-turn conversation without tool calling
- ToolAgentLoop: multi-turn conversation with tool calling, interaction

To use ToolAgentLoop, user only need to provide tools configuration in json/yaml file. In the configuration file, user should specify following fields for each tool:
- class_name: fully qualified class name of the tool used to dynamically load the custom tool class
- config: key-word arguments used to initialize the tool instance

Let's dump our sandbox tool configuration to a json file:

In [6]:
# ray.shutdown()

sandbox = Sandbox.remote()
sandbox_address = ray.get(sandbox.get_server_address.remote())

tool_config = {
    "tools": [
        {
            "class_name": "sandbox.SandboxTool",
            "config": {
                "type": "native",
                "sandbox_fusion_url": f"http://{sandbox_address}/run_code",
            },
        },
    ],
}

tool_config_path = "tool_config.json"
with open(tool_config_path, "w") as f:
    json.dump(tool_config, f)

In [7]:
from hydra import compose, initialize_config_dir

with initialize_config_dir(config_dir=verl_config_dir):
    config = compose(
        config_name="ppo_trainer",
        overrides=[
            "algorithm.adv_estimator=grpo",
            "data.train_files=" + train_file,
            "data.val_files=" + test_file,
            "data.return_raw_chat=True",
            "data.train_batch_size=32",
            "data.max_prompt_length=1024",
            "data.max_response_length=1024",
            "+data.apply_chat_template_kwargs.enable_thinking=False",
            # actor related
            "actor_rollout_ref.model.path=" + model_path,
            "actor_rollout_ref.actor.ppo_mini_batch_size=8",
            "actor_rollout_ref.actor.ppo_micro_batch_size_per_gpu=8",
            "actor_rollout_ref.actor.fsdp_config.param_offload=True",
            "actor_rollout_ref.actor.fsdp_config.optimizer_offload=True",
            # rollout related
            "actor_rollout_ref.rollout.name=" + rollout_name,
            "actor_rollout_ref.rollout.mode=async",
            "actor_rollout_ref.rollout.tensor_model_parallel_size=1",
            "actor_rollout_ref.rollout.n=8",
            "actor_rollout_ref.rollout.multi_turn.tool_config_path=" + tool_config_path,
            "actor_rollout_ref.rollout.agent.default_agent_loop=tool_agent",
            "actor_rollout_ref.rollout.log_prob_micro_batch_size_per_gpu=8",
            # trainer related
            "trainer.val_before_train=True",
            "trainer.log_val_generations=10",
            "trainer.n_gpus_per_node=2",
            "trainer.test_freq=-1",
            "trainer.total_training_steps=2",
            "trainer.logger=['console', 'wandb']",
            "trainer.project_name=verl",
            "trainer.experiment_name=" + os.path.basename(model_path),
        ],
    )

/tmp/ipykernel_389590/3529998727.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(config_dir=verl_config_dir):


In [8]:
from verl.trainer.main_ppo import main

main(config)

(TaskRunner pid=397737, ip=10.122.252.202) TaskRunner hostname: n122-252-202, PID: 397737
(TaskRunner pid=397737, ip=10.122.252.202) {'actor_rollout_ref': {'actor': {'_target_': 'verl.workers.config.FSDPActorConfig',
(TaskRunner pid=397737, ip=10.122.252.202)                                  'calculate_entropy': False,
(TaskRunner pid=397737, ip=10.122.252.202)                                  'checkpoint': {'_target_': 'verl.trainer.config.CheckpointConfig',
(TaskRunner pid=397737, ip=10.122.252.202)                                                 'async_save': False,
(TaskRunner pid=397737, ip=10.122.252.202)                                                 'load_contents': ['model',
(TaskRunner pid=397737, ip=10.122.252.202)                                                                   'optimizer',
(TaskRunner pid=397737, ip=10.122.252.202)                                                                   'extra'],
(TaskRunner pid=397737, ip=10.122.252.202)                       

(TaskRunner pid=397737, ip=10.122.252.202) /opt/tiger/verl_context_folding/verl/trainer/main_ppo.py:302: UserWarning: Disabled critic as algorithm.adv_estimator != gae. If it is not intended, please set critic.enable=True
(TaskRunner pid=397737, ip=10.122.252.202)   use_critic=need_critic(config),
(TaskRunner pid=397737, ip=10.122.252.202) /opt/tiger/verl_context_folding/verl/utils/profiler/config.py:49: UserWarning: Torch profiler tool config is not fully supported now.
(TaskRunner pid=397737, ip=10.122.252.202)   warnings.warn("Torch profiler tool config is not fully supported now.", stacklevel=1)


(TaskRunner pid=397737, ip=10.122.252.202) Using dataset class: RLHFDataset
(TaskRunner pid=397737, ip=10.122.252.202) {
(TaskRunner pid=397737, ip=10.122.252.202)   "type": "function",
(TaskRunner pid=397737, ip=10.122.252.202)   "function": {
(TaskRunner pid=397737, ip=10.122.252.202)     "name": "code_interpreter",
(TaskRunner pid=397737, ip=10.122.252.202)     "description": "Execute the code in the sandbox.",
(TaskRunner pid=397737, ip=10.122.252.202)     "parameters": {
(TaskRunner pid=397737, ip=10.122.252.202)       "type": "object",
(TaskRunner pid=397737, ip=10.122.252.202)       "properties": {
(TaskRunner pid=397737, ip=10.122.252.202)         "code": {
(TaskRunner pid=397737, ip=10.122.252.202)           "type": "string",
(TaskRunner pid=397737, ip=10.122.252.202)           "description": "The code to be executed."
(TaskRunner pid=397737, ip=10.122.252.202)         }
(TaskRunner pid=397737, ip=10.122.252.202)       },
(TaskRunner pid=397737, ip=10.122.252.202)       "requi

Generating train split: 100 examples [00:00, 25324.86 examples/s]
(TaskRunner pid=397737, ip=10.122.252.202) /opt/tiger/verl_context_folding/verl/trainer/ppo/ray_trainer.py:327: UserWarning: Disabled critic as algorithm.adv_estimator != gae. If it is not intended, please set critic.enable=True
(TaskRunner pid=397737, ip=10.122.252.202)   self.use_critic = need_critic(self.config)


(WorkerDict pid=401424, ip=10.122.252.202) Skip starting flight recorder server because No module named 'bytedance.ndtimeline'.
(WorkerDict pid=401425, ip=10.122.252.202) Skip starting flight recorder server because No module named 'bytedance.ndtimeline'.
(WorkerDict pid=401424, ip=10.122.252.202) Model config after override: Qwen3Config {
(WorkerDict pid=401424, ip=10.122.252.202)   "architectures": [
(WorkerDict pid=401424, ip=10.122.252.202)     "Qwen3ForCausalLM"
(WorkerDict pid=401424, ip=10.122.252.202)   ],
(WorkerDict pid=401424, ip=10.122.252.202)   "attention_bias": false,
(WorkerDict pid=401424, ip=10.122.252.202)   "attention_dropout": 0.0,
(WorkerDict pid=401424, ip=10.122.252.202)   "eos_token_id": 151645,
(WorkerDict pid=401424, ip=10.122.252.202)   "head_dim": 128,
(WorkerDict pid=401424, ip=10.122.252.202)   "hidden_act": "silu",
(WorkerDict pid=401424, ip=10.122.252.202)   "hidden_size": 2048,
(WorkerDict pid=401424, ip=10.122.252.202)   "initializer_range": 0.02,
(Wo

...

(vLLMHttpServer pid=402300, ip=10.122.252.202) INFO 12-12 06:44:13 [async_llm.py:269] Added request 9189841de7614e76ba20e2a9c81e4632.
(vLLMHttpServer pid=402297, ip=10.122.252.202) INFO 12-12 06:44:13 [async_llm.py:269] Added request 3de6eca3ff4a49c4b44c13f750a0baaf.
(vLLMHttpServer pid=402300, ip=10.122.252.202) INFO 12-12 06:44:13 [async_llm.py:269] Added request 7e9b1cd532b64e438b83888477d13c6e.
(vLLMHttpServer pid=402297, ip=10.122.252.202) INFO 12-12 06:44:13 [async_llm.py:269] Added request 586ebc3acdd24496b2dac24b377e2b96.
(vLLMHttpServer pid=402300, ip=10.122.252.202) INFO 12-12 06:44:13 [async_llm.py:269] Added request f62f77f35089424fbdd6479d9039a2ad.
(vLLMHttpServer pid=402300, ip=10.122.252.202) INFO 12-12 06:44:14 [async_llm.py:269] Added request 774df79d033c47a5beb9dfb9a7021f8e.
(vLLMHttpServer pid=402300, ip=10.122.252.202) INFO 12-12 06:44:14 [async_llm.py:269] Added request e05dd61bc82b491e9ab8c9454b10cad8.
(vLLMHttpServer pid=402300, ip=10.122.252.202) INFO 12-12 06:4

(AgentLoopWorker pid=403390, ip=10.122.252.202) ERROR:2025-12-12 06:44:15,079:Failed to decode tool call: Invalid control character at: line 2 column 1660 (char 1660)
(AgentLoopWorker pid=403400, ip=10.122.252.202) ERROR:2025-12-12 06:44:15,754:Failed to decode tool call: Expecting ',' delimiter: line 2 column 119 (char 119)


(vLLMHttpServer pid=402300, ip=10.122.252.202) INFO 12-12 06:44:22 [async_llm.py:606] Engines are idle, requests have been drained
(vLLMHttpServer pid=402300, ip=10.122.252.202) INFO 12-12 06:44:22 [block_pool.py:321] Successfully reset prefix cache
(vLLMHttpServer pid=402297, ip=10.122.252.202) INFO 12-12 06:44:22 [async_llm.py:606] Engines are idle, requests have been drained
(vLLMHttpServer pid=402297, ip=10.122.252.202) INFO 12-12 06:44:22 [block_pool.py:321] Successfully reset prefix cache
(WorkerDict pid=401424, ip=10.122.252.202) INFO 12-12 06:44:22 [gpu_worker.py:104] Sleep mode freed 34.79 GiB memory, 1.46 GiB memory is still in use.
(WorkerDict pid=401425, ip=10.122.252.202) INFO 12-12 06:44:22 [gpu_worker.py:104] Sleep mode freed 34.61 GiB memory, 1.46 GiB memory is still in use.
(TaskRunner pid=397737, ip=10.122.252.202) step:2 - actor/entropy:0.20166516304016113 - actor/pg_clipfrac:np.float64(0.0014586237502953736) - actor/ppo_kl:np.float64(-6.779428728975745e-05) - actor/

Training Progress: 100%|██████████| 2/2 [01:21<00:00, 40.58s/it]
(TaskRunner pid=397737, ip=10.122.252.202) wandb: Waiting for W&B process to finish... (success).
(TaskRunner pid=397737, ip=10.122.252.202) wandb: 
(TaskRunner pid=397737, ip=10.122.252.202) wandb: Run history:
(TaskRunner pid=397737, ip=10.122.252.202) wandb:                                            actor/entropy █▁
(TaskRunner pid=397737, ip=10.122.252.202) wandb:                                          actor/grad_norm █▁
(TaskRunner pid=397737, ip=10.122.252.202) wandb:                                                 actor/lr ▁▁
(TaskRunner pid=397737, ip=10.122.252.202) wandb:                                        actor/pg_clipfrac ▁█
(TaskRunner pid=397737, ip=10.122.252.202) wandb:                                  actor/pg_clipfrac_lower ▁█
(TaskRunner pid=397737, ip=10.122.252.202) wandb:                                            actor/pg_loss ▁█
(TaskRunner pid=397737, ip=10.122.252.202) wandb:              

(raylet, ip=None) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff4b819ed7a015d273690feee301000000 Worker ID: cdaf68f29a2c457df2348b307aed0cb95f5aecbd504b1e13fa7b1daa Node ID: 240a606d653dbcae4fd999c95a0a93e6632e0a239b421ab1f2496ec0 Worker IP address: 10.122.252.202 Worker port: 43901 Worker PID: 405709 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly by a signal. SystemExit is raised (sys.exit is called). Exit code: 1. The process receives a SIGTERM.


For demo purpose, we only train 5 steps, you can verify the training process by checking wandb metrics:
- num_turns: min/max/mean chat conversation turns in each step.
- critic rewards: min/max/mean critic rewards in each step.

For more realistic agentic RL training, please refer to our recipe:
- [retool](https://github.com/volcengine/verl/tree/main/recipe/retool): implementation of paper [ReTool: Reinforcement Learning for Strategic Tool Use in LLMs](https://arxiv.org/abs/2504.11536)
- [collabllm](https://github.com/volcengine/verl/tree/main/recipe/collabllm): implementation of paper [CollabLLM: From Passive Responders to Active Collaborators](https://arxiv.org/pdf/2502.00640)
- [deepeyes](https://github.com/volcengine/verl/tree/main/recipe/deepeyes): implementation of paper [DeepEyes: Incentivizing "Thinking with Images" via Reinforcement Learning](https://arxiv.org/abs/2505.14362)